In [1]:
import pandas as pd
import numpy as np
import os
from collections import deque

pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)

# Day 3 Gear Ratios
You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:

467..114..\
...*......\
..35..633.\
......#...\
617*......\
.....+.58.\
..592.....\
......755.\
...$.*....\
.664.598..\

In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

In [2]:
with open('input.txt', 'r') as file:
    input = file.read()

# input = '''467..114..
# ...*......
# ..35..633.
# ......#...
# 617*......
# .....+.58.
# ..592.....
# ......755.
# ...$.*....
# .664.598..'''

DF = pd.DataFrame(pd.Series(input.split('\n')).apply(list).to_list()).dropna()
PART_NUMBERS = {}

In [3]:
class Tile:
    def __init__(self, i, j):
        self.i = i
        self.j = j

    def get(self):
        return DF.iloc[self.i, self.j]
    
    def check_number(self):
        if self.get().isdigit():
            return True
        return False
    
## create a class to manage checking surroundings
class Map:
    def __init__(self, i, j):
        self.c = Tile(i, j)
        self.r = Tile(i, j+1)
        self.l = Tile(i, j-1)
        self.u = Tile(i-1, j)
        self.d = Tile(i+1, j)
        self.ur = Tile(i-1, j+1)
        self.ul = Tile(i-1, j-1)
        self.dr = Tile(i+1, j+1)
        self.dl = Tile(i+1, j-1)
    

def add_part_number(m: Map):
    part_dqueue = deque([m.c.get()])
    temp_m = m

    # check left tiles
    while temp_m.l.check_number():
        part_dqueue.appendleft(temp_m.l.get())
        temp_m = Map(temp_m.l.i, temp_m.l.j)
    
    temp_m = m

    # check right tiles
    while temp_m.r.check_number():
        part_dqueue.append(temp_m.r.get())
        temp_m = Map(temp_m.r.i, temp_m.r.j)

    part_number = int(''.join(map(str, part_dqueue)))

    # check if part is already in dict
    if (temp_m.c.i, temp_m.c.j) not in PART_NUMBERS.keys():
        print(f'Found Part Number -> ({temp_m.c.i}, {temp_m.c.j}): {part_number}')
        PART_NUMBERS[(temp_m.c.i, temp_m.c.j)] = part_number

    
def check_around_map(m: Map):
    try:
        if m.r.check_number():
            add_part_number(Map(m.r.i, m.r.j))
    except IndexError:
        pass

    try:
        if m.l.check_number():
            add_part_number(Map(m.l.i, m.l.j))
    except IndexError:
        pass
    
    try:
        if m.u.check_number():
            add_part_number(Map(m.u.i, m.u.j))
    except IndexError:
        pass

    try:
        if m.d.check_number():
            add_part_number(Map(m.d.i, m.d.j))
    except IndexError:
        pass
    
    try:
        if m.ur.check_number():
            add_part_number(Map(m.ur.i, m.ur.j))
    except IndexError:
        pass

    try:
        if m.ul.check_number():
            add_part_number(Map(m.ul.i, m.ul.j))
    except IndexError:
        pass
    
    try:
        if m.dl.check_number():
            add_part_number(Map(m.dl.i, m.dl.j))
    except IndexError:
        pass

    try:
        if m.dr.check_number():
            add_part_number(Map(m.dr.i, m.dr.j))
    except IndexError:
        pass

def check_current_map(m: Map):
    if m.c.get() != '.' and not m.c.get().isdigit():
        return True
    return False



In [4]:
for i in range(len(DF)):
    for j in range(len(DF.columns)):
        current_map = Map(i, j)
        
        # check if we are at a special character
        if check_current_map(current_map):
            check_around_map(current_map)

Found Part Number -> (2, 12): 360
Found Part Number -> (0, 11): 662
Found Part Number -> (0, 17): 71
Found Part Number -> (0, 48): 805
Found Part Number -> (2, 50): 677
Found Part Number -> (0, 61): 862
Found Part Number -> (2, 60): 780
Found Part Number -> (0, 65): 680
Found Part Number -> (1, 78): 92
Found Part Number -> (2, 109): 539
Found Part Number -> (1, 22): 811
Found Part Number -> (1, 35): 846
Found Part Number -> (1, 40): 855
Found Part Number -> (3, 40): 379
Found Part Number -> (1, 75): 230
Found Part Number -> (3, 75): 969
Found Part Number -> (2, 101): 959
Found Part Number -> (3, 97): 976
Found Part Number -> (1, 131): 92
Found Part Number -> (3, 133): 579
Found Part Number -> (4, 24): 809
Found Part Number -> (4, 31): 482
Found Part Number -> (2, 30): 664
Found Part Number -> (2, 44): 881
Found Part Number -> (4, 44): 492
Found Part Number -> (4, 53): 650
Found Part Number -> (2, 56): 934
Found Part Number -> (4, 63): 38
Found Part Number -> (4, 86): 106
Found Part Num

In [8]:
len(PART_NUMBERS.values())

1060